In [1]:
import tensorflow as tf
from keras.models import load_model
from keras.models import Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, Activation
import numpy as np 
from skimage.measure import compare_ssim
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import keras
import random
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras_ex.gkernel import GaussianKernel, GaussianKernel2, GaussianKernel3
from keras.utils import to_categorical
from math import log10, sqrt 
import cv2 
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

Using TensorFlow backend.


In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
def flatten(imgs):
    return np.array([im.flatten() for im in imgs])

def computing_SSIM(individual,target):
    target = target.reshape(28,28)
    individual = individual.reshape(28,28)
    if individual.shape == target.shape:
        (score, diff) = compare_ssim(individual, target, full=True)
    else:
        print(target.shape)
        print(individual.shape)
    return score

In [4]:
# load flatten and normalize mnist
(x_train, y), (x_test, y_t) = tf.keras.datasets.mnist.load_data()
X = flatten(x_train)
X_test = flatten(x_test)
X_n = np.array([x/255. for x in X])
X_t_n = np.array([x/255. for x in X_test])

In [5]:
file_ = open('subset_cnn', 'rb')
original = pickle.load(file_)
adversarial = pickle.load(file_)
original_y = pickle.load(file_)
adversarial_y = pickle.load(file_)
file_.close()

In [6]:
file_ = open('subset_rbf', 'rb')
original_rbf = pickle.load(file_)
adversarial_rbf = pickle.load(file_)
original_y_rbf = pickle.load(file_)
adversarial_y_rbf = pickle.load(file_)
file_.close()

In [7]:
file_ = open('adv_rbf_1_qua', 'rb')
adversarials = pickle.load(file_)
times = pickle.load(file_)
nfes  = pickle.load(file_)
psnrs = pickle.load(file_)
ssim_scores = pickle.load(file_)
predicted_classes = pickle.load(file_)
file_.close()

In [8]:
inp = Input(shape=(28,28,1))
conv1 = Conv2D(32, (3,3),activation='relu')(inp)
conv2 = Conv2D(32, (3,3),activation='relu')(conv1)
max_pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
drop = Dropout(0.25)(max_pool1)
flat =  Flatten()(drop)
dense1 = Dense(128,activation="relu")(flat)
drop = Dropout(0.5)(dense1)
predictions = Dense(10,activation="softmax")(drop)
model = Model(inputs=[inp], outputs=[predictions])
model.compile(optimizer='Adadelta', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.load_weights('CNN_MNIST.h5')
#print(model)
#model.summary()

In [9]:
model_preds = np.argmax(model.predict(np.array(original).reshape(np.array(original).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset:', accuracy_score(model_preds, original_y))
model_preds = np.argmax(model.predict(np.array(adversarial).reshape(np.array(adversarial).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y))
model_preds = np.argmax(model.predict(np.array(adversarials).reshape(np.array(adversarials).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on adversarials:', accuracy_score(model_preds, adversarial_y))
sub = np.concatenate((original, np.array(adversarials).reshape(np.array(adversarials).shape[0],784)))
sub_y = np.concatenate((original_y, adversarial_y))
model_preds = np.argmax(model.predict(np.array(sub).reshape(np.array(sub).shape[0], 28, 28, 1)/255.), axis=1)
print('accuracy on subset with adversarials:',accuracy_score(model_preds, sub_y))

accuracy on subset: 0.9888268156424581
accuracy on adversarial: 0.9920634920634921
accuracy on adversarials: 0.873015873015873
accuracy on subset with adversarials: 0.9745347698334965


In [10]:
(X, y), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X = X.reshape((X.shape[0], -1))
X_sc = X / 255.0
X_sc.shape
y_cat = to_categorical(y)
X_test = X_test.reshape((X_test.shape[0], -1))
X_test_sc = X_test / 255.0

num_lm0 = 100
num_lm = num_lm0 * 10
init_list = []
for ii in range(10):
    init_wgt0 = X[y==ii]
    init_wgt0 = init_wgt0[np.random.choice(range(init_wgt0.shape[0]), size=num_lm0, replace=False)] + \
                np.random.normal(scale=0.01, size=num_lm0*784).reshape(num_lm0, 784)
    init_list.append(init_wgt0)
init_wgt = np.vstack(init_list)
init_wgt = init_wgt[np.random.permutation(range(init_wgt.shape[0]))]

inp2 = Input(shape=(28*28,), name='inp')
oup2 = GaussianKernel(num_lm, 28*28,
                     kernel_gamma='auto', weights=[init_wgt],
                     name='gkernel1')(inp2)
oup2 = Dense(10, activation='softmax')(oup2)
model2 = Model(inp2, oup2)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.load_weights("rbfnn_v2.h5")

In [11]:
model_preds = np.argmax(model2.predict(np.array(original_rbf).reshape(np.array(original_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on subset:', accuracy_score(model_preds, original_y_rbf))
model_preds = np.argmax(model2.predict(np.array(adversarial_rbf).reshape(np.array(adversarial_rbf).shape[0], -1)/255.), axis=1)
print('accuracy on adversarial:', accuracy_score(model_preds, adversarial_y_rbf))
model_preds = np.argmax(model2.predict(np.array(adversarials).reshape(np.array(adversarials).shape[0], -1)/255.), axis=1)
print('accuracy on adversarials:', accuracy_score(model_preds, adversarial_y_rbf))
sub = np.concatenate((original_rbf, np.array(adversarials).reshape(np.array(adversarials).shape[0],784)))
sub_y = np.concatenate((original_y_rbf, adversarial_y_rbf))
model_preds = np.argmax(model2.predict(np.array(sub).reshape(np.array(sub).shape[0], -1)/255.), axis=1)
print('accuracy on subset with adversarials:',accuracy_score(model_preds, sub_y))

accuracy on subset: 0.9363128491620112
accuracy on adversarial: 0.9365079365079365
accuracy on adversarials: 0.2619047619047619
accuracy on subset with adversarials: 0.8550440744368266
